In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# -*- encoding: utf-8 -*-
import os
import pymetis as metis
from geopy.distance import geodesic
import pickle
import queue
import sys
from tqdm import tqdm
import datetime

In [2]:
edge_infos = pd.read_csv('data/chengdu/edges.csv')
edge_infos

,Unnamed: 0,osmid,highway,oneway,length,name,geometry,lanes,bridge,ref,junction,maxspeed,tunnel,u,v,key,fid
0,0,28623889,unclassified,False,20.517,NaN,"LINESTRING (104.100294 30.7147948, 104.1000959...",NaN,NaN,NaN,NaN,NaN,NaN,288416374,1815728532,0,0
1,1,137133120,primary,True,143.316,昭觉寺南路,"LINESTRING (104.100294 30.7147948, 104.1007853...",NaN,NaN,NaN,NaN,NaN,NaN,288416374,3541376393,0,1
2,2,28623889,unclassified,False,173.857,NaN,"LINESTRING (104.100294 30.7147948, 104.1015994...",NaN,NaN,NaN,NaN,NaN,NaN,288416374,314655436,0,2
3,3,184255152,primary,True,277.305,解放路一段,"LINESTRING (104.083862 30.6898358, 104.0828949...",3,NaN,NaN,NaN,NaN,NaN,288416379,1159231411,0,3
4,4,184296328,tertiary,False,318.437,平福巷,"LINESTRING (104.083862 30.6898358, 104.086253 ...",NaN,NaN,NaN,NaN,NaN,NaN,288416379,1947583836,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6397,6397,887890886,secondary,True,13.047,王贾路,"LINESTRING (104.066007 30.7199718, 104.0660103...",NaN,NaN,NaN,NaN,NaN,NaN,8255138229,8255138230,0,6397
6398,6398,92408253,primary,True,11.466,中环路洞子口路段,"LINESTRING (104.0660103 30.7200891, 104.065913...",NaN,NaN,NaN,NaN,NaN,NaN,8255138230,1902099198,0,6398
6399,6399,887890886,secondary,True,65.922,王贾路,"LINESTRING (104.0660103 30.7200891, 104.066022...",NaN,NaN,NaN,NaN,NaN,NaN,8255138230,8255138232,0,6399
6400,6400,887890887,secondary,True,57.625,王贾路,"LINESTRING (104.0659598 30.7206667, 104.065913...",NaN,NaN,NaN,NaN,NaN,NaN,8255138232,1902099198,0,6400


In [3]:
edge_csv = 'data/chengdu/match_20161002'
edge_pd = pd.read_csv(open(edge_csv),encoding='utf-8', sep='\t', names=['record'])
edge_pd

,record
0,"([(73905.0, 1351, 0.001426821498687921, 0.0023..."
1,"([(93874.0, 10, 0.0014642188614543141, 0.00166..."
2,"([(83799.0, 94588907833792, 0.0009953947996033..."
3,"([(64698.0, 94588873610704, 3.891465522005535e..."
4,"([(115747.0, 886, 0.0018644497740566348, 0.001..."
...,...
187804,"([(121266.0, 148, 0.0013835595870487396, 0.001..."
187805,"([(100505.0, 2682, 0.001809829788175622, 0.006..."
187806,"([(82634.0, 57, 0.005577467102277626, 0.029929..."
187807,"([(114386.0, 3777, 0.0028879300224176445, 0.00..."


In [15]:
# 单条道路计算
speedlimit_dict = {'bridleway':10, 'construction':0, 'cycleway':15, 'footway':5, 'living_street':5,
'motorway':50,'motorway_link':50,'path':5,'pedestrian':5,'platform':5,'primary':40,'primary_link':40,
'raceway':30,'residential':5,'road':10,'secondary':30,'secondary_link':30,'service':10,'steps':5,
'tertiary':10,'tertiary_link':10,'track':5,'trunk':50,'trunk_link':50,'unclassified':20,'subway':50,'rail':40}

'''
([(22263.0, 94588319464960, 4.3213290315087855e-05, 0.0019982740577713415), (22266.0, 3061, 0.000123562414790473, 0.0019982740577713415)], [3061])
'''
start_timestamp = datetime.datetime(2016, 10, 1, 0, 0, 0).timestamp()

delta_time_length = 60 # 1 min

trajs_records = []
limit_size = 100000
cnt,file_cnt = 0,0

for index, row in edge_pd.iterrows():
    token = row['record']
    data = token[2:-2].split("], [")
    if len(data) != 2:
        continue

    trajs, links = data
    trajs = [p.split(",") for p in trajs[1:-1].split("), (")]
    links = links.split(",")
    last_link = None
    last_t, last_mile = None, None

    for traj in trajs:

        delta_t, linkid, delta_dist, link_dist = traj
        timestamp = float(start_timestamp)+float(delta_t)
        linkid = int(linkid)
        delta_dist = float(delta_dist)
        link_dist = float(link_dist)

        if last_link == None or last_link != linkid:
            pass
        else:
            try:
                ty = edge_pd.iloc[last_link]['highway']
            except Exception as err:
                ty = "none"
            limitspeed = speedlimit_dict.get(ty, 50)/3.6
            speed = (delta_dist - last_mile)*111000/(float(delta_t) - float(last_t))
            if speed < limitspeed * 2:
                trajs_records.append({'link':linkid,'tsp':timestamp//delta_time_length*delta_time_length,'speed':speed})
                cnt+=1
                if cnt> limit_size:
                    cnt = 0
                    file_cnt+=1

        # trajs_counts.append({'link':linkid,'tsp':timestamp//delta_time_length*delta_time_length})
              
        last_link = linkid
        last_t = delta_t
        last_mile = delta_dist

KeyboardInterrupt: 

In [36]:
edge_pd

,record
0,"([(73905.0, 1351, 0.001426821498687921, 0.0023..."
1,"([(93874.0, 10, 0.0014642188614543141, 0.00166..."
2,"([(83799.0, 94588907833792, 0.0009953947996033..."
3,"([(64698.0, 94588873610704, 3.891465522005535e..."
4,"([(115747.0, 886, 0.0018644497740566348, 0.001..."
...,...
187804,"([(121266.0, 148, 0.0013835595870487396, 0.001..."
187805,"([(100505.0, 2682, 0.001809829788175622, 0.006..."
187806,"([(82634.0, 57, 0.005577467102277626, 0.029929..."
187807,"([(114386.0, 3777, 0.0028879300224176445, 0.00..."


In [42]:

file_list

['match_20161012',
 'match_20161124',
 'match_20161129',
 'match_20161126',
 'match_20161016',
 'match_20161122',
 'match_20161001',
 'match_20161114',
 'match_20161123',
 'match_20161116',
 'match_20161007',
 'match_20161005',
 'match_20161004',
 'match_20161130',
 'match_20161115',
 'match_20161013',
 'match_20161101',
 'match_20161008',
 'match_20161113',
 'match_20161021',
 'match_20161006',
 'match_20161017',
 'match_20161125',
 'match_20161103',
 'match_20161024',
 'match_20161028',
 'match_20161106',
 'match_20161030',
 'match_20161119',
 'match_20161117',
 'match_20161026',
 'match_20161110',
 'match_20161102',
 'match_20161121',
 'match_20161019',
 'match_20161105',
 'match_20161002',
 'match_20161112',
 'match_20161025',
 'match_20161010',
 'match_20161128',
 'match_20161018',
 'match_20161111',
 'match_20161023',
 'match_20161020',
 'match_20161022',
 'match_20161120',
 'match_20161009',
 'match_20161107',
 'match_20161104',
 'match_20161003',
 'match_20161011',
 'match_2016

In [3]:
speedlimit_dict = {'bridleway':10, 'construction':0, 'cycleway':15, 'footway':5, 'living_street':5,
'motorway':50,'motorway_link':50,'path':5,'pedestrian':5,'platform':5,'primary':40,'primary_link':40,
'raceway':30,'residential':5,'road':10,'secondary':30,'secondary_link':30,'service':10,'steps':5,
'tertiary':10,'tertiary_link':10,'track':5,'trunk':50,'trunk_link':50,'unclassified':20,'subway':50,'rail':40}
'''
([(22263.0, 94588319464960, 4.3213290315087855e-05, 0.0019982740577713415), (22266.0, 3061, 0.000123562414790473, 0.0019982740577713415)], [3061])
'''
start_timestamp = datetime.datetime(2016, 10, 1, 0, 0, 0).timestamp()
delta_time_length = 60 # 1 min
edge_infos = pd.read_csv('data/chengdu/edges.csv')

folder_path = 'data/chengdu/'

file_list = [file for file in os.listdir(folder_path) if file.startswith('match')]

for edge_csv in file_list:

# edge_csv = 'data/chengdu/match_20161002'
    edge_pd = pd.read_csv(open(folder_path+edge_csv),encoding='utf-8', sep='\t', names=['record'])

    rst = []
    for index, row in tqdm(edge_pd.iterrows()):
        token = row['record']
        data = token[2:-2].split("], [")
        if len(data) != 2:
            continue
        trajs, links = data
        trajs = [p.split(",") for p in trajs[1:-1].split("), (")]
        links = links.split(",")
        last_link = None
        last_points = []

        for traj in trajs:
            delta_t, linkid, delta_dist, link_dist = traj
            timestamp = start_timestamp+float(delta_t)
            linkid = int(linkid)
            delta_dist = float(delta_dist)

            if last_link != linkid:
                if last_link != None:
                    if len(last_points)>1:
                        start = last_points[0]
                        end =last_points[-1]

                        time = end[0] - start[0]
                        dist = end[1] - start[1]
                        if time <=0:
                            last_points = []
                            last_link = linkid
                            continue
                        try:
                            ty = edge_pd.iloc[last_link]['highway']
                        except Exception as err:
                            ty = "none"
                        limitspeed = speedlimit_dict.get(ty, 50)/3.6

                        avg_speed = dist * 110000 / time 
                        if avg_speed< limitspeed*2:
                            for t in range(int(start[0]//delta_time_length*delta_time_length),int(end[0]//delta_time_length*delta_time_length)+1,delta_time_length):
                                rst.append({'speed':avg_speed, 'time':t, 'link':linkid})
                    last_points = [] 
                last_link = linkid

            last_points.append(( timestamp, delta_dist))
    
    rst_df = pd.DataFrame(rst)
    rst_df.to_csv('data/chengdu/60/'+edge_csv+'_60.csv') 

182748it [03:50, 791.62it/s]
204941it [04:13, 808.19it/s]
214540it [04:29, 796.96it/s]
219083it [04:33, 802.15it/s]
142659it [02:59, 796.00it/s]
201142it [04:07, 812.99it/s]
206998it [04:29, 767.14it/s]
195474it [04:00, 813.78it/s]
204325it [04:17, 792.25it/s]
208198it [04:12, 824.62it/s]
180007it [03:52, 775.04it/s]
180007it [03:45, 798.26it/s]
179183it [03:37, 822.73it/s] 
214952it [03:55, 911.84it/s] 
200825it [03:26, 970.87it/s] 
149792it [02:36, 959.99it/s] 
181105it [03:05, 977.46it/s] 
172913it [03:01, 952.37it/s] 
190520it [03:20, 948.17it/s] 
207951it [03:54, 887.70it/s]
188528it [03:30, 896.79it/s]
138580it [02:27, 937.64it/s] 
220641it [03:47, 967.88it/s] 
188689it [03:45, 836.44it/s]
194446it [03:58, 813.98it/s] 
202121it [03:41, 913.45it/s] 
190250it [03:51, 823.23it/s]
188111it [03:47, 827.84it/s] 
222141it [03:50, 964.03it/s] 
207185it [03:23, 1019.01it/s]
194322it [03:27, 935.99it/s] 
195204it [03:25, 950.18it/s] 
186187it [03:11, 974.78it/s] 
197953it [03:25, 962.29it/

In [4]:
speedlimit_dict = {'bridleway':10, 'construction':0, 'cycleway':15, 'footway':5, 'living_street':5,
'motorway':50,'motorway_link':50,'path':5,'pedestrian':5,'platform':5,'primary':40,'primary_link':40,
'raceway':30,'residential':5,'road':10,'secondary':30,'secondary_link':30,'service':10,'steps':5,
'tertiary':10,'tertiary_link':10,'track':5,'trunk':50,'trunk_link':50,'unclassified':20,'subway':50,'rail':40}
'''
([(22263.0, 94588319464960, 4.3213290315087855e-05, 0.0019982740577713415), (22266.0, 3061, 0.000123562414790473, 0.0019982740577713415)], [3061])
'''
# start_timestamp = datetime.datetime(2016, 10, 1, 0, 0, 0).timestamp()
delta_time_length = 300 # 1 min
edge_infos = pd.read_csv('data/chengdu/edges.csv')

folder_path = 'data/chengdu/'

file_list = [file for file in os.listdir(folder_path) if file.startswith('match')]

for edge_csv in file_list:
    edge_pd = pd.read_csv(open(folder_path+edge_csv),encoding='utf-8', sep='\t', names=['record'])

    rst = []
    for index, row in tqdm(edge_pd.iterrows()):
        token = row['record']
        data = token[2:-2].split("], [")
        if len(data) != 2:
            continue
        trajs, links = data
        trajs = [p.split(",") for p in trajs[1:-1].split("), (")]
        links = links.split(",")
        last_link = None
        last_points = []

        for traj in trajs:
            delta_t, linkid, delta_dist, link_dist = traj
            timestamp = delta_t
            linkid = int(linkid)
            delta_dist = float(delta_dist)

            if last_link != linkid:
                if last_link != None:
                    if len(last_points)>1:
                        start = last_points[0]
                        end =last_points[-1]

                        time = end[0] - start[0]
                        dist = end[1] - start[1]
                        if time <=0:
                            last_points = []
                            last_link = linkid
                            continue
                        try:
                            ty = edge_pd.iloc[last_link]['highway']
                        except Exception as err:
                            ty = "none"
                        limitspeed = speedlimit_dict.get(ty, 50)/3.6

                        avg_speed = dist * 110000 / time 
                        if avg_speed< limitspeed*2:
                            for t in range(int(start[0]//delta_time_length*delta_time_length),int(end[0]//delta_time_length*delta_time_length)+1,delta_time_length):
                                rst.append({'speed':avg_speed, 'time':t, 'link':linkid})
                    last_points = [] 
                last_link = linkid

            last_points.append(( timestamp, delta_dist))
    
    rst_df = pd.DataFrame(rst)
    rst_df.to_csv('data/chengdu/300/'+edge_csv+'_300.csv') 

182748it [04:10, 728.96it/s]
204941it [04:50, 705.65it/s]
214540it [04:44, 753.94it/s]
219083it [04:52, 748.98it/s]
142659it [03:15, 729.81it/s]
201142it [04:43, 710.34it/s]
206998it [05:06, 676.07it/s]
195474it [04:20, 751.40it/s]
204325it [04:41, 725.96it/s]
208198it [04:33, 761.44it/s]
180007it [04:01, 745.12it/s]
180007it [04:00, 747.41it/s]
179183it [04:15, 701.21it/s]
214952it [04:51, 737.24it/s]
200825it [04:16, 781.73it/s] 
149792it [03:23, 736.79it/s]
181105it [04:10, 722.74it/s]
172913it [04:00, 717.53it/s]
190520it [04:17, 739.42it/s]
207951it [04:56, 700.91it/s]
188528it [05:15, 598.42it/s]
138580it [03:13, 716.21it/s]
220641it [04:51, 757.77it/s]
188689it [03:28, 905.92it/s] 
194446it [03:41, 879.73it/s] 
202121it [03:52, 868.46it/s] 
190250it [03:32, 894.13it/s] 
188111it [03:24, 919.02it/s] 
222141it [03:53, 951.67it/s] 
207185it [03:49, 903.80it/s] 
194322it [03:35, 901.01it/s] 
195204it [03:32, 918.90it/s] 
186187it [03:20, 930.47it/s] 
197953it [03:28, 949.11it/s] 
12

In [5]:
edge_pd

,record
0,"([(3362845.0, 926, 0.0023585231184868817, 0.00..."
1,"([(3411169.0, 5587, 0.005664714293271909, 0.01..."
2,"([(3355981.0, 94588501313376, 0.00088133179119..."
3,"([(3373726.0, 94588642815696, 0.00096259876862..."
4,"([(3383345.0, 967, 6.736025141568123e-05, 0.00..."
...,...
194338,"([(3358755.0, 3450, 0.0005018545933500198, 0.0..."
194339,"([(3401278.0, 926, 0.0021096704509555213, 0.00..."
194340,"([(3388522.0, 3296, 0.005272782218779342, 0.00..."
194341,"([(3401558.0, 382, 0.0003623245196724546, 0.00..."


In [32]:
df_avg = rst_df.groupby(['link', 'time'])['speed'].mean().reset_index()

# 使用 pivot_table 将数据转换为行（link）和列（time）的格式
df_pivot = df_avg.pivot(index='link', columns='time', values='speed')

In [33]:
df_pivot

time,1.475323e+09,1.475323e+09,1.475323e+09,1.475323e+09,1.475324e+09,1.475324e+09,1.475324e+09,1.475324e+09,1.475324e+09,1.475324e+09,...,1.475409e+09,1.475409e+09,1.475409e+09,1.475409e+09,1.475409e+09,1.475409e+09,1.475409e+09,1.475410e+09,1.475410e+09,1.475410e+09
link,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.945397,...,12.292975,11.934389,10.233791,NaN,10.542762,13.525344,12.446036,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.686529,NaN,NaN,13.393827,NaN,4.166477,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10.466626,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6399,NaN,NaN,NaN,NaN,NaN,17.567903,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6.518991,NaN,NaN,NaN,NaN


In [35]:
rst_df.link.max()

6401